In [1]:
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import date
from collections import namedtuple

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import percent as P

In [3]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"),
      pd.get_option("display.max_rows"))

40 200


# Initial Setting

In [4]:
def ctg(nml, cld):
    idx = ['nml', 'cld']
    return Series([nml, cld], index = idx)

In [5]:
# Pickling Data
with open("data/area.pickle", "rb") as fr:
    data_area = pickle.load(fr)
areadf = data_area['areadf']

In [6]:
"""
# Call out the area data
area_rent = areadf.loc[[x in data_area['rent_col'] for x in areadf.name]]
area_rent = area_rent.area_py.groupby(areadf['floor']).sum()
area_cld = round(area_rent[['b1', 'f1']].sum(), 2)
area_nml = round(area_rent[['f2', 'f3', 'f4']].sum(), 2)
"""

"\n# Call out the area data\narea_rent = areadf.loc[[x in data_area['rent_col'] for x in areadf.name]]\narea_rent = area_rent.area_py.groupby(areadf['floor']).sum()\narea_cld = round(area_rent[['b1', 'f1']].sum(), 2)\narea_nml = round(area_rent[['f2', 'f3', 'f4']].sum(), 2)\n"

In [7]:
# Call out the area data
data_mtrxpy = data_area['mtrxpy']
loc_cld = [x[1] in ['b1', 'f1'] for x in data_mtrxpy.index]
loc_nml = [x[1] in ['f2', 'f3', 'f4'] for x in data_mtrxpy.index]
area_cld = round(data_mtrxpy[('rent', 'sum')].loc[loc_cld].sum(), 2)
area_nml = round(data_mtrxpy[('rent', 'sum')].loc[loc_nml].sum(), 2)

### Valuation Class

In [8]:
class Valuation:
    def __init__(self, rent, mtnc, cap_rate, area, vcncy_rate, oprtg_rate, IR, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate = cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.__intlz__()
        self.ctgr = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate',
                     'rent_amt', 'mtnc_amt', 'dpst_amt', 'IR_amt',
                     'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value']
        self.ctgr_rate = ['cap_rate', 'vcncy_rate', 'oprtg_rate']
        

    def __intlz__(self):
        self.dpst_amt = self.rent * 10 * self.area / self.RU # deposit amount
        self.rent_amt = self.rent * 12 * self.area / self.RU # yearly rent amount
        self.mtnc_amt = self.mtnc * 12 * self.area / self.RU # yearly maintenance cost
        self.IR_amt = self.dpst_amt * self.IR
        
        self.PGI = self.IR_amt + self.rent_amt + self.mtnc_amt
        self.EGI = self.PGI * (1 - self.vcncy_rate)
        self.oprtg_cst = self.PGI * self.oprtg_rate
        self.NOI = self.EGI - self.oprtg_cst
        self.value = self.NOI / self.cap_rate
        self.value_sum = self.value.sum()
        
    def printval(self, withsum=True):
        tmpdct = DataFrame({x: getattr(self, x) for x in self.ctgr})
        if withsum:
            tmpdct_withsum = tmpdct.append(DataFrame([tmpdct.sum()], index=['sum']))
            return DataFrame(tmpdct_withsum)
        else:
            return DataFrame(tmpdct)

In [9]:
class Val_Mtrx:
    def __init__(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None, RU=1_000_000):
        self.rent = rent
        self.mtnc = mtnc
        self.cap_rate=cap_rate
        self.area = area
        self.vcncy_rate = vcncy_rate
        self.oprtg_rate = oprtg_rate
        self.IR = IR
        self.RU = RU
        
        self.ctgr = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate',
                     'rent_amt', 'mtnc_amt', 'dpst_amt', 'IR_amt',
                     'PGI', 'EGI', 'oprtg_cst', 'NOI', 'value']
        self.ctgr_rate = ['cap_rate', 'vcncy_rate', 'oprtg_rate']
        self.ctgr_cld = ['nml', 'cld']
        
        self.rslt_lst = []
    
    
    @property
    def ctgr_rate_mlti(self):
        ctgr_rslt = []
        for val1 in self.ctgr_rate:
            for val2 in self.ctgr_cld:
                ctgr_rslt.append((val1, val2))
        return ctgr_rslt
    
        
    def value(self, rent=None, mtnc=None, cap_rate=None, area=None, 
                 vcncy_rate=None, oprtg_rate=None, IR=None):
        keys = ['rent', 'mtnc', 'cap_rate', 'area', 'vcncy_rate', 'oprtg_rate', 'IR']
        dct = {}
        for key in keys:
            if locals()[key] is not None:
                tmp_val = locals()[key]
            else:
                tmp_val = getattr(self, key)
            dct['tmp_'+key] = tmp_val
            
        tmpistnc = Valuation(rent = dct['tmp_rent'],
                             mtnc = dct['tmp_mtnc'],
                             cap_rate = dct['tmp_cap_rate'],
                             area = dct['tmp_area'],
                             vcncy_rate = dct['tmp_vcncy_rate'],
                             oprtg_rate = dct['tmp_oprtg_rate'],
                             IR = dct['tmp_IR'])
        return tmpistnc

                
    def mkdict(self, **kwargs):
        lenarg = len(kwargs)
        keylst = []
        arglst = []
        for key, item in kwargs.items():
            keylst.append(key)
            arglst.append(item)
        
        for val0 in arglst[0]:
            if lenarg == 1:
                tmpdct = {keylst[0]:val0}
                self.rslt_lst.append(self.value(**tmpdct))
            else:
                for val1 in arglst[1]:
                    if lenarg == 2:
                        tmpdct = {keylst[0]:val0, keylst[1]:val1}
                        self.rslt_lst.append(self.value(**tmpdct))
                    else:
                        for val2 in arglst[2]:
                            if lenarg == 3:
                                tmpdct = {keylst[0]:val0, keylst[1]:val1, keylst[2]:val2}
                                self.rslt_lst.append(self.value(**tmpdct))

    
    def printlst(self):
        tmplst = []
        tmpsum = []
        for rslt in self.rslt_lst:
            tmprslt = rslt.printval(withsum=False).unstack()
            tmplst.append(tmprslt)
            
            tmpsum.append(rslt.value_sum)
        tmpdf = DataFrame(tmplst)
        tmpdf[('value', 'sum')] = tmpsum
        return tmpdf

# Valuation
### 1) Assumption

In [10]:
val_mtrx = Val_Mtrx(rent = ctg(23_000, 50_000),
                    cap_rate = ctg(P(4.2), P(5.0)),
                    mtnc = ctg(2_000, 2_000),
                    area = ctg(area_nml, area_cld),
                    vcncy_rate = ctg(0.0, 0.0),
                    oprtg_rate = P(14.5),
                    IR = P(1.0))

In [11]:
rent_case = [ctg(21_000, 46_000), # unit rent
             ctg(22_000, 48_000),
             ctg(23_000, 50_000),
             ctg(24_000, 52_000),
             ctg(25_000, 54_000),
             ctg(26_000, 56_000)]
cap_rate = [ctg(P(4.0), P(4.6)),
            ctg(P(4.1), P(4.8)),
            ctg(P(4.2), P(5.0)),
            ctg(P(4.3), P(5.2)),
            ctg(P(4.4), P(5.4))]

val_mtrx.mkdict(rent=rent_case, cap_rate=cap_rate)
val_mtrx.rslt_lst

### 2) Pickling Result

In [12]:
data_valuation = {}

In [13]:
valuation_mtrx = val_mtrx.printlst()
data_valuation['valuation_mtrx'] = valuation_mtrx
R(valuation_mtrx, rate=val_mtrx.ctgr_rate_mlti)

rent           mtnc        cap_rate         area        vcncy_rate  \
       nml     cld    nml    cld      nml   cld    nml    cld        nml   
0   21,000  46,000  2,000  2,000     4.00  4.60  8,370  6,855       0.00   
1   21,000  46,000  2,000  2,000     4.10  4.80  8,370  6,855       0.00   
2   21,000  46,000  2,000  2,000     4.20  5.00  8,370  6,855       0.00   
3   21,000  46,000  2,000  2,000     4.30  5.20  8,370  6,855       0.00   
4   21,000  46,000  2,000  2,000     4.40  5.40  8,370  6,855       0.00   
5   22,000  48,000  2,000  2,000     4.00  4.60  8,370  6,855       0.00   
6   22,000  48,000  2,000  2,000     4.10  4.80  8,370  6,855       0.00   
7   22,000  48,000  2,000  2,000     4.20  5.00  8,370  6,855       0.00   
8   22,000  48,000  2,000  2,000     4.30  5.20  8,370  6,855       0.00   
9   22,000  48,000  2,000  2,000     4.40  5.40  8,370  6,855       0.00   
10  23,000  50,000  2,000  2,000     4.00  4.60  8,370  6,855       0.00   
11  23,000  50,000  2,000  2,000     4.10  4.80  8,370  6,855       0.00   
12  23,000  50,000  2,000  2,000     4.20  5.00  8,370  6,855       0.00   
13  23,000  50,000  2,000  2,000     4.30  5.20  8,370  6,855       0.00   
14  23,000  50,000  2,000  2,000     4.40  5.40  8,370  6,855       0.00   
15  24,000  52,000  2,000  2,000     4.00  4.60  8,370  6,855       0.00   
16  24,000  52,000  2,000  2,000     4.10  4.80  8,370  6,855       0.00   
17  24,000  52,000  2,000  2,000     4.20  5.00  8,370  6,855       0.00   
18  24,000  52,000  2,000  2,000     4.30  5.20  8,370  6,855       0.00   
19  24,000  52,000  2,000  2,000     4.40  5.40  8,370  6,855       0.00   
20  25,000  54,000  2,000  2,000     4.00  4.60  8,370  6,855       0.00   
21  25,000  54,000  2,000  2,000     4.10  4.80  8,370  6,855       0.00   
22  25,000  54,000  2,000  2,000     4.20  5.00  8,370  6,855       0.00   
23  25,000  54,000  2,000  2,000     4.30  5.20  8,370  6,855       0.00   
24  25,000  54,000  2,000  2,000     4.40  5.40  8,370  6,855       0.00   
25  26,000  56,000  2,000  2,000     4.00  4.60  8,370  6,855       0.00   
26  26,000  56,000  2,000  2,000     4.10  4.80  8,370  6,855       0.00   
27  26,000  56,000  2,000  2,000     4.20  5.00  8,370  6,855       0.00   
28  26,000  56,000  2,000  2,000     4.30  5.20  8,370  6,855       0.00   
29  26,000  56,000  2,000  2,000     4.40  5.40  8,370  6,855       0.00   

         oprtg_rate        rent_amt        mtnc_amt      dpst_amt         \
     cld        nml    cld      nml    cld      nml  cld      nml    cld   
0   0.00      14.50  14.50    2,109  3,784      201  165    1,758  3,153   
1   0.00      14.50  14.50    2,109  3,784      201  165    1,758  3,153   
2   0.00      14.50  14.50    2,109  3,784      201  165    1,758  3,153   
3   0.00      14.50  14.50    2,109  3,784      201  165    1,758  3,153   
4   0.00      14.50  14.50    2,109  3,784      201  165    1,758  3,153   
5   0.00      14.50  14.50    2,210  3,948      201  165    1,841  3,290   
6   0.00      14.50  14.50    2,210  3,948      201  165    1,841  3,290   
7   0.00      14.50  14.50    2,210  3,948      201  165    1,841  3,290   
8   0.00      14.50  14.50    2,210  3,948      201  165    1,841  3,290   
9   0.00      14.50  14.50    2,210  3,948      201  165    1,841  3,290   
10  0.00      14.50  14.50    2,310  4,113      201  165    1,925  3,427   
11  0.00      14.50  14.50    2,310  4,113      201  165    1,925  3,427   
12  0.00      14.50  14.50    2,310  4,113      201  165    1,925  3,427   
13  0.00      14.50  14.50    2,310  4,113      201  165    1,925  3,427   
14  0.00      14.50  14.50    2,310  4,113      201  165    1,925  3,427   
15  0.00      14.50  14.50    2,410  4,277      201  165    2,009  3,565   
16  0.00      14.50  14.50    2,410  4,277      201  165    2,009  3,565   
17  0.00      14.50  14.50    2,410  4,277      201  165    2,009  3,565   
18  0.00      14.50  14.50    2,410  4,277      201  

In [14]:
valuation_smry = val_mtrx.printlst()[['rent', 'cap_rate', 'value']]
data_valuation['valuation_smry'] = valuation_smry
R(valuation_smry, rate=val_mtrx.ctgr_rate_mlti)

rent         cap_rate         value                 
       nml     cld      nml   cld     nml     cld      sum
0   21,000  46,000     4.00  4.60  49,753  73,975  123,728
1   21,000  46,000     4.10  4.80  48,539  70,893  119,432
2   21,000  46,000     4.20  5.00  47,384  68,057  115,441
3   21,000  46,000     4.30  5.20  46,282  65,440  111,721
4   21,000  46,000     4.40  5.40  45,230  63,016  108,246
5   22,000  48,000     4.00  4.60  51,917  77,059  128,976
6   22,000  48,000     4.10  4.80  50,651  73,848  124,499
7   22,000  48,000     4.20  5.00  49,445  70,894  120,339
8   22,000  48,000     4.30  5.20  48,295  68,167  116,463
9   22,000  48,000     4.40  5.40  47,198  65,643  112,840
10  23,000  50,000     4.00  4.60  54,082  80,142  134,224
11  23,000  50,000     4.10  4.80  52,763  76,803  129,566
12  23,000  50,000     4.20  5.00  51,507  73,731  125,238
13  23,000  50,000     4.30  5.20  50,309  70,895  121,204
14  23,000  50,000     4.40  5.40  49,166  68,269  117,435
15  24,000  52,000     4.00  4.60  56,247  83,226  139,472
16  24,000  52,000     4.10  4.80  54,875  79,758  134,633
17  24,000  52,000     4.20  5.00  53,568  76,568  130,136
18  24,000  52,000     4.30  5.20  52,323  73,623  125,945
19  24,000  52,000     4.40  5.40  51,133  70,896  122,029
20  25,000  54,000     4.00  4.60  58,412  86,309  144,720
21  25,000  54,000     4.10  4.80  56,987  82,713  139,700
22  25,000  54,000     4.20  5.00  55,630  79,404  135,034
23  25,000  54,000     4.30  5.20  54,336  76,350  130,687
24  25,000  54,000     4.40  5.40  53,101  73,522  126,624
25  26,000  56,000     4.00  4.60  60,576  89,392  149,969
26  26,000  56,000     4.10  4.80  59,099  85,668  144,766
27  26,000  56,000     4.20  5.00  57,692  82,241  139,933
28  26,000  56,000     4.30  5.20  56,350  79,078  135,428
29  26,000  56,000     4.40  5.40  55,069  76,149  131,218

In [15]:
valuation_pivot = val_mtrx.printlst()[['rent', 'cap_rate', 'value']]
col_name = [x + "_" + y for x, y in valuation_pivot.columns]
valuation_pivot.columns = col_name
valuation_pivot['rent'] = list(zip([f"{x:,.0f}" for x in valuation_pivot['rent_nml']], 
                                   [f"{x:,.0f}" for x in valuation_pivot['rent_cld']]))
valuation_pivot['cap_rate'] = list(zip([round(x * 100, 1) for x in valuation_pivot['cap_rate_nml']], 
                                       [round(x * 100, 1) for x in valuation_pivot['cap_rate_cld']]))
valuation_pivot = valuation_pivot.pivot_table(index='rent', columns='cap_rate', values='value_sum')
data_valuation['valuation_pivot'] = valuation_pivot
valuation_pivot

cap_rate,"(4.0, 4.6)","(4.1, 4.8)","(4.2, 5.0)","(4.3, 5.2)","(4.4, 5.4)"
rent,,,,,
"(21,000, 46,000)",123728.145875,119432.354807,115440.933387,111721.398016,108245.846723
"(22,000, 48,000)",128976.234392,124499.171513,120339.270379,116462.686070,112840.346931
"(23,000, 50,000)",134224.322910,129565.988219,125237.607370,121203.974124,117434.847139
"(24,000, 52,000)",139472.411427,134632.804925,130135.944362,125945.262177,122029.347347
"(25,000, 54,000)",144720.499944,139699.621632,135034.281354,130686.550231,126623.847555
"(26,000, 56,000)",149968.588462,144766.438338,139932.618346,135427.838285,131218.347763


In [16]:
with open('data/valuation.pickle', 'wb') as fw:
    pickle.dump(data_valuation, fw)

### 3) Exceling

In [17]:
data_valuation['valuation_mtrx'].to_excel("data/valuation_mtrx.xlsx")
data_valuation['valuation_pivot'].to_excel("data/valuation_pivot.xlsx")

### 4) Exercising